# Practical Example

## High dimension

The practical example consists of the stable space estimation for a multivariate time series that contains the information of 205 economic indicators since January 2017 

In [1]:
# packages
remove(list = ls())
options(warn = -1)
suppressMessages(library(data.table))
suppressMessages(library(dplyr))
suppressMessages(library(tidyr))
suppressMessages(library(ggplot2))
suppressMessages(library(xtable))

source("../source/simulations.R")
source("../source/vectorial_methods.R")
source("../source/auxiliar_methods.R")

## Several econonomic indicators in Mexico

In [2]:
# Variables BIE
dt_BIE <- data.table::fread("../databases/variables_BIE.csv")
catalogue_BIE <- data.table::fread("../databases/catalogue_BIE.csv")

We centered the columns of the series, and we get the auxiliary blocks for the methods

In [3]:
X_BIE <- as.matrix(dt_BIE, rownames = "Date")
rownames(X_BIE) <- NULL
# X_BIE <- scale(ts(X_BIE))
XX_BIE <- X_BIE[1:(nrow(X_BIE)-1),]
Y_BIE <- X_BIE[2:nrow(X_BIE),]

We check the order of integration of each variable. We count the number of series between the $m$ series that are $\textsf{I}(0)$, $\textsf{I}(1)$ and $\textsf{I}(2)$

In [4]:
aux <- X_BIE

# Step 1: Collect p-values for level, first difference, second difference
pvals_level <- apply(aux, 2, function(x) tryCatch(tseries::kpss.test(x)$p.value, error = function(e) NA))
pvals_diff1 <- apply(aux, 2, function(x) tryCatch(tseries::kpss.test(diff(x))$p.value, error = function(e) NA))
pvals_diff2 <- apply(aux, 2, function(x) tryCatch(tseries::kpss.test(diff(diff(x)))$p.value, error = function(e) NA))

# Step 2: Classify integration order
I0 <- which(pvals_level  > 0.05)
I1 <- which(pvals_level <= 0.05 & pvals_diff1 > 0.05)
I2 <- which(pvals_level <= 0.05 & pvals_diff1 <= 0.05 & pvals_diff2 > 0.05)

# Step 3: Report counts
cat("Number of I(0) series:", length(I0), "\n")
cat("Number of I(1) series:", length(I1), "\n")
cat("Number of I(2) series:", length(I2), "\n")


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



Number of I(0) series: 185 
Number of I(1) series: 19 
Number of I(2) series: 1 


We estimate the stable space for this multivariate time series using the methods
- PCA
- SPCA
- PLS

In [5]:
basis_BIE_PLS <- basis_stable(X_BIE, method = "pls")
basis_BIE_PCA <- basis_stable(scale(X_BIE),method="pca")
# SPCA
spca_para <- 1
basis_BIE_SPCA <- basis_stable(scale(X_BIE),method = "spca", 
                                        spca_engine = "elasticnet",
                                        spca_sparse = "penalty",
                                        spca_para = spca_para)


We check the estimated stable space dimension

In [6]:
cat("The PLS estimated dimension is: ", ncol(basis_BIE_PLS$basis_S))
cat("\nThe PCA estimated dimension is: ", ncol(basis_BIE_PCA$basis_S))
cat("\nThe SPCA estimated dimension is: ", ncol(basis_BIE_SPCA$basis_S))

The PLS estimated dimension is:  191
The PCA estimated dimension is:  196
The SPCA estimated dimension is:  81

In [7]:
vecs_PLS <- pls_alg(X_BIE)
vecs_PCA <- pca_alg(scale(X_BIE))
vecs_SPCA <- spca_alg(scale(X_BIE), engine = "elasticnet", 
                                    sparse = "penalty", para = spca_para)

In [28]:
dim(basis_BIE_PCA$basis_S)

[1] 205 196

In [25]:
dim(vecs_SPCA)

[1] 205 205

## Projection Error

We calculate the proyection onto the first two stable components. We also consider the first two principal components.

In [8]:
# PLS
T_scores_PLS <- XX_BIE%*%basis_BIE_PLS$weights_S 
Y_est_PLS <- scores_rebuilt(T_scores_PLS[,1:2],Y=Y_BIE)

# PCA
T_scores_PCA <- XX_BIE%*%basis_BIE_PCA$basis_S
Y_est_PCA <- scores_rebuilt(T_scores_PCA[,1:2],Y=Y_BIE)

# SPCA
T_scores_SPCA <- XX_BIE%*%basis_BIE_SPCA$basis_S
Y_est_SPCA <- scores_rebuilt(T_scores_SPCA[,1:2],Y=Y_BIE)

# PCA with the first two non-stable components
T_scores_PCA_non_stable <- XX_BIE%*%basis_BIE_PCA$basis_N[,1:2]
Y_est_PCA_non_stable <- scores_rebuilt(T_scores_PCA_non_stable[,1:2],Y=Y_BIE)

# PLS with the first two non-stable components
T_scores_PLS_non_stable <- XX_BIE%*%basis_BIE_PLS$basis_N[,1:2]
Y_est_PLS_non_stable <- scores_rebuilt(T_scores_PLS_non_stable[,1:2],Y=Y_BIE)

# SPCA with the first two non-stable components
T_scores_SPCA_non_stable <- XX_BIE%*%basis_BIE_SPCA$basis_N[,1:2]
Y_est_SPCA_non_stable <- scores_rebuilt(T_scores_SPCA_non_stable[,1:2],Y=Y_BIE)

We could calculate the normalized MSE for each serie

In [9]:
MSE <- matrix(0,ncol(Y_BIE),6) 
colnames(MSE) <- c("non-stable PCA","stationary PCA",
                   "non-stable PLS","stationary PLS",
                   "non-stable SPCA", "stationary SPCA")
rownames(MSE) <- colnames(Y_BIE)

# error estimation
MSE[,"stationary PLS"] <- VNMSE_(Y_BIE,Y_est_PLS$est)
MSE[,"stationary PCA"] <- VNMSE_(Y_BIE,Y_est_PCA$est)
MSE[,"non-stable PCA"] <- VNMSE_(Y_BIE,Y_est_PCA_non_stable$est)
MSE[,"non-stable PLS"] <- VNMSE_(Y_BIE,Y_est_PLS_non_stable$est)
MSE[,"stationary SPCA"] <- VNMSE_(Y_BIE,Y_est_SPCA$est)
MSE[,"non-stable SPCA"] <- VNMSE_(Y_BIE,Y_est_SPCA_non_stable$est)

Print the results of the estimation using the first two non-stable components and the first two stable components.

In [10]:
print(xtable(MSE[c("s23",head(names(I1),3),names(I2)),],digits = 2))

% latex table generated in R 4.1.2 by xtable 1.8-4 package
% Sat Sep  6 11:01:41 2025
\begin{table}[ht]
\centering
\begin{tabular}{rrrrrrr}
  \hline
 & non-stable PCA & stationary PCA & non-stable PLS & stationary PLS & non-stable SPCA & stationary SPCA \\ 
  \hline
s23 & 0.30 & 0.43 & 0.36 & 0.44 & 0.30 & 0.42 \\ 
  imaief\_Hgo & 0.43 & 0.64 & 0.42 & 0.60 & 0.42 & 0.63 \\ 
  imaief\_SLP & 0.52 & 0.54 & 0.53 & 0.54 & 0.51 & 0.54 \\ 
  s54 & 0.52 & 0.58 & 0.63 & 0.61 & 0.52 & 0.56 \\ 
  rem\_cmenor & 0.30 & 0.44 & 0.45 & 0.47 & 0.29 & 0.42 \\ 
   \hline
\end{tabular}
\end{table}


We choose two series $\textsf{I}(1)$ for visualisation

In [11]:
options(repr.plot.width = 9, repr.plot.height = 6)

In [12]:
dates <- dt_BIE[,Date][2:nrow(dt_BIE)]

# Color-blind safe palette (Okabe-Ito + black for observed)
custom_palette <- c(
  "stationary PLS"  = "#56B4E9",  # sky blue
  "stationary PCA"  = "#009E73",  # bluish green
  "Observed"        = "black",    # black
  "stationary SPCA" = "#CC79A7",  # reddish purple
  
  # Non-stable components (newly adjusted to Okabe-Ito safe colours)
  "non-stable PCA"  = "#0072B2",  # blue
  "non-stable PLS"  = "#D55E00",  # vermillion
  "non-stable SPCA" = "#999999"   # grey
)

# Linetypes for better distinction
custom_linetypes <- c(
  "Observed"        = "solid",
  "stationary PCA"  = "dotdash",
  "stationary PLS"  = "dotted",
  "stationary SPCA" = "twodash",
  # Non-stable components (completing your request)
  "non-stable PCA"  = "dashed",
  "non-stable PLS"  = "dotdash",
  "non-stable SPCA" = "dotted"
)


gg_methods_fit <- plot_estimates_comparison(
                    Y_obs = Y_BIE,
                    Y_est_list = list(`stationary PLS` = Y_est_PLS$est, `stationary PCA` = Y_est_PCA$est, `non-stable PCA` = Y_est_PCA_non_stable$est,
                    `non-stable PLS` = Y_est_PLS_non_stable$est, `stationary SPCA` = Y_est_SPCA$est, `non-stable SPCA` = Y_est_SPCA_non_stable$est),
                    col_names = c("s23","s54"),
                    labels = c("stationary PLS", "stationary PCA", "stationary SPCA",
                                "non-stable PCA", "non-stable PLS", "non-stable SPCA"),  # optional if list has names
                    dates = dates,
                    method_palette = custom_palette,
                    method_linetypes = custom_linetypes
                  )

ggsave(
  filename = "../images/Figure_6.pdf",
  plot = gg_methods_fit,
  device = "pdf",
  width = 12,   # scale by relative widths (e.g., 1.45 units × 4 inches per unit)
  height = 8,         # set a reasonable height (adjust if needed)
  units = "in", dpi = 300)